In [1]:
import pandas as pd
import os
import numpy as np
import librosa
from sklearn.preprocessing import OneHotEncoder

In [2]:
paths=[]
labels=[]
for filename in os.listdir('./AudioWAV'):
    
    paths.append('./AudioWAV/' + filename)
    file = filename.split('.')[0]
   
    label = file.split('_')[2]
    if label == 'ANG':
        labels.append('angry.wav')
    elif label == 'DIS':
        labels.append('disgust.wav')
    elif label == 'FEA':
        labels.append('fear.wav')
    elif label == 'HAP':
        labels.append('happy.wav')
    elif label == 'NEU':
        labels.append('neutral.wav')
    elif label == 'SAD':
        labels.append('sad.wav')
        

df_cremad = pd.DataFrame({'speech':paths,'label':labels})
df_cremad.sample(5)

,speech,label
5086,./AudioWAV/1063_IEO_SAD_MD.wav,sad.wav
3179,./AudioWAV/1040_DFA_ANG_XX.wav,angry.wav
1662,./AudioWAV/1021_IWL_DIS_XX.wav,disgust.wav
5438,./AudioWAV/1067_IWL_SAD_XX.wav,sad.wav
3435,./AudioWAV/1043_IEO_DIS_LO.wav,disgust.wav


In [3]:
def MFCC(filename):
    y, sr = librosa.load(filename,duration=3,offset=0.5)
    return np.mean(librosa.feature.mfcc(y=y,sr=sr,n_mfcc=40).T,axis=0)

mfcc_cremad = df_cremad['speech'].apply(lambda x:MFCC(x))

In [4]:
X =[x for x in mfcc_cremad]
X =np.array(X)
X.shape
X =np.expand_dims(X,-1)
X.shape

(7442, 40, 1)

In [5]:
ohe=OneHotEncoder()
y = ohe.fit_transform(df_cremad[['label']] )
y = y.toarray()

In [6]:
X.shape, y.shape

((7442, 40, 1), (7442, 6))

In [7]:
df_cremad['label'].unique()

array(['angry.wav', 'disgust.wav', 'fear.wav', 'happy.wav', 'neutral.wav',
       'sad.wav'], dtype=object)

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np

# Definindo os modelos

class SimpleDNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(SimpleDNN, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(input_size, 128)
        self.relu = nn.ReLU()
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 64)
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

class LSTMAttention(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMAttention, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.attention = nn.Linear(hidden_size, 1)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        out, _ = self.lstm(x, (h0, c0))
        attn_weights = torch.softmax(self.attention(out), dim=1)
        out = torch.sum(attn_weights * out, dim=1)
        out = self.fc(out)
        return out

class CNNModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(64, 32, kernel_size=3, padding=1)
        self.fc_input_size = 32 * 1 * 1
        self.fc1 = nn.Linear(self.fc_input_size, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.dropout(x)
        x = x.view(-1, self.fc_input_size)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


class CNNAttention(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNNAttention, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(64, 32, kernel_size=3, padding=1)
        self.attention = nn.Linear(32, 1)
        self.fc = nn.Linear(32, num_classes)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        attn_weights = self.softmax(self.attention(x.permute(0, 2, 1))).squeeze(-1)
        attn_weights = attn_weights.unsqueeze(-1)
        x = torch.sum(attn_weights * x, dim=2)
        x = self.fc(x)
        return x
    
# Construindo e treinando os modelos

input_size = X.shape[1:]
num_classes = y.shape[1]
hidden_size = 64
num_layers = 2
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Convertendo os dados para tensores PyTorch
X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
y_tensor = torch.tensor(y, dtype=torch.float32).to(device)

# Definindo o tamanho do lote
batch_size = 32

# Criando conjuntos de dados PyTorch
dataset = torch.utils.data.TensorDataset(X_tensor, y_tensor)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Definindo tensor para o LSTM
X_tensorLSTM = X_tensor.permute(0, 2, 1)

# Criando conjuntos de dados PyTorch para LSTM
datasetLSTM = torch.utils.data.TensorDataset(X_tensorLSTM, y_tensor)
train_sizeLSTM = int(0.8 * len(datasetLSTM))
test_sizeLSTM = len(datasetLSTM) - train_sizeLSTM
train_datasetLSTM, test_datasetLSTM = torch.utils.data.random_split(datasetLSTM, [train_sizeLSTM, test_sizeLSTM])

# DataLoader para o LSTM
train_loaderLSTM = DataLoader(train_datasetLSTM, batch_size=batch_size, shuffle=True)
test_loaderLSTM = DataLoader(test_datasetLSTM, batch_size=batch_size, shuffle=False)

# Convertendo os dados para tensores PyTorch do CNN
X_tensorCNN = torch.tensor(X, dtype=torch.float32).to(device)

# Criando conjuntos de dados PyTorch do CNN
datasetCNN = torch.utils.data.TensorDataset(X_tensorCNN, y_tensor)
train_sizeCNN = int(0.8 * len(datasetCNN))
test_sizeCNN = len(datasetCNN) - train_sizeCNN
train_datasetCNN, test_datasetCNN = torch.utils.data.random_split(datasetCNN, [train_sizeCNN, test_sizeCNN])

# DataLoader para o CNN
train_loaderCNN = DataLoader(train_datasetCNN, batch_size=batch_size, shuffle=True)
test_loaderCNN = DataLoader(test_datasetCNN, batch_size=batch_size, shuffle=False)

# Função para treinamento
def train(model, train_loader, criterion, optimizer, num_epochs=1000):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            #print(outputs.shape)
            loss = criterion(outputs, torch.max(labels, 1)[1])
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == torch.max(labels, 1)[1]).sum().item()

        epoch_loss = running_loss / len(train_loader)
        accuracy = correct / total
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.4f}')

In [9]:
# Standard Deep Neural Network
sdnn_model = SimpleDNN(input_size[0], num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(sdnn_model.parameters(), lr=0.0001)
train(sdnn_model, train_loader, criterion, optimizer)

Epoch [1/1000], Loss: 10.0616, Accuracy: 0.1634
Epoch [2/1000], Loss: 4.3335, Accuracy: 0.1660
Epoch [3/1000], Loss: 2.6167, Accuracy: 0.1730
Epoch [4/1000], Loss: 2.1857, Accuracy: 0.1641
Epoch [5/1000], Loss: 1.9942, Accuracy: 0.1702
Epoch [6/1000], Loss: 1.9260, Accuracy: 0.1678
Epoch [7/1000], Loss: 1.8830, Accuracy: 0.1678
Epoch [8/1000], Loss: 1.8415, Accuracy: 0.1717
Epoch [9/1000], Loss: 1.8302, Accuracy: 0.1769
Epoch [10/1000], Loss: 1.8257, Accuracy: 0.1735
Epoch [11/1000], Loss: 1.8226, Accuracy: 0.1710
Epoch [12/1000], Loss: 1.8128, Accuracy: 0.1712
Epoch [13/1000], Loss: 1.8102, Accuracy: 0.1702
Epoch [14/1000], Loss: 1.8036, Accuracy: 0.1712
Epoch [15/1000], Loss: 1.8023, Accuracy: 0.1713
Epoch [16/1000], Loss: 1.8069, Accuracy: 0.1683
Epoch [17/1000], Loss: 1.7989, Accuracy: 0.1745
Epoch [18/1000], Loss: 1.7979, Accuracy: 0.1722
Epoch [19/1000], Loss: 1.7979, Accuracy: 0.1754
Epoch [20/1000], Loss: 1.7963, Accuracy: 0.1737
Epoch [21/1000], Loss: 1.7969, Accuracy: 0.1715


In [10]:
# LSTM
lstm_model = LSTMModel(input_size[0], hidden_size, num_layers, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(lstm_model.parameters(), lr=0.0001)
train(lstm_model, train_loaderLSTM, criterion, optimizer)

Epoch [1/1000], Loss: 1.7842, Accuracy: 0.1729
Epoch [2/1000], Loss: 1.7523, Accuracy: 0.2903
Epoch [3/1000], Loss: 1.6796, Accuracy: 0.3200
Epoch [4/1000], Loss: 1.6100, Accuracy: 0.3187
Epoch [5/1000], Loss: 1.5794, Accuracy: 0.3299
Epoch [6/1000], Loss: 1.5664, Accuracy: 0.3397
Epoch [7/1000], Loss: 1.5528, Accuracy: 0.3477
Epoch [8/1000], Loss: 1.5362, Accuracy: 0.3534
Epoch [9/1000], Loss: 1.5253, Accuracy: 0.3588
Epoch [10/1000], Loss: 1.5224, Accuracy: 0.3660
Epoch [11/1000], Loss: 1.5106, Accuracy: 0.3669
Epoch [12/1000], Loss: 1.5046, Accuracy: 0.3743
Epoch [13/1000], Loss: 1.4961, Accuracy: 0.3805
Epoch [14/1000], Loss: 1.4914, Accuracy: 0.3843
Epoch [15/1000], Loss: 1.4808, Accuracy: 0.3880
Epoch [16/1000], Loss: 1.4784, Accuracy: 0.3909
Epoch [17/1000], Loss: 1.4714, Accuracy: 0.3906
Epoch [18/1000], Loss: 1.4619, Accuracy: 0.3966
Epoch [19/1000], Loss: 1.4594, Accuracy: 0.3990
Epoch [20/1000], Loss: 1.4535, Accuracy: 0.3961
Epoch [21/1000], Loss: 1.4529, Accuracy: 0.4015
E

In [11]:
# LSTM with Attention
lstm_atn_model = LSTMAttention(input_size[0], hidden_size, num_layers, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(lstm_atn_model.parameters(), lr=0.0001)
train(lstm_atn_model, train_loaderLSTM, criterion, optimizer)

Epoch [1/1000], Loss: 1.7823, Accuracy: 0.2201
Epoch [2/1000], Loss: 1.7445, Accuracy: 0.2752
Epoch [3/1000], Loss: 1.6775, Accuracy: 0.3192
Epoch [4/1000], Loss: 1.6269, Accuracy: 0.3177
Epoch [5/1000], Loss: 1.5888, Accuracy: 0.3250
Epoch [6/1000], Loss: 1.5729, Accuracy: 0.3366
Epoch [7/1000], Loss: 1.5587, Accuracy: 0.3397
Epoch [8/1000], Loss: 1.5505, Accuracy: 0.3477
Epoch [9/1000], Loss: 1.5388, Accuracy: 0.3494
Epoch [10/1000], Loss: 1.5263, Accuracy: 0.3534
Epoch [11/1000], Loss: 1.5265, Accuracy: 0.3602
Epoch [12/1000], Loss: 1.5209, Accuracy: 0.3706
Epoch [13/1000], Loss: 1.5138, Accuracy: 0.3711
Epoch [14/1000], Loss: 1.5068, Accuracy: 0.3759
Epoch [15/1000], Loss: 1.5034, Accuracy: 0.3780
Epoch [16/1000], Loss: 1.4919, Accuracy: 0.3845
Epoch [17/1000], Loss: 1.4926, Accuracy: 0.3805
Epoch [18/1000], Loss: 1.4791, Accuracy: 0.3850
Epoch [19/1000], Loss: 1.4807, Accuracy: 0.3924
Epoch [20/1000], Loss: 1.4687, Accuracy: 0.3961
Epoch [21/1000], Loss: 1.4663, Accuracy: 0.3986
E

In [12]:
# Convolutional Neural Network
cnn_model = CNNModel(input_size[0],num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model.parameters(), lr=0.0001)
train(cnn_model, train_loaderCNN, criterion, optimizer)

Epoch [1/1000], Loss: 1.8564, Accuracy: 0.1697
Epoch [2/1000], Loss: 1.7609, Accuracy: 0.2229
Epoch [3/1000], Loss: 1.7086, Accuracy: 0.2666
Epoch [4/1000], Loss: 1.6702, Accuracy: 0.2844
Epoch [5/1000], Loss: 1.6403, Accuracy: 0.3015
Epoch [6/1000], Loss: 1.6248, Accuracy: 0.3037
Epoch [7/1000], Loss: 1.6085, Accuracy: 0.3148
Epoch [8/1000], Loss: 1.5981, Accuracy: 0.3177
Epoch [9/1000], Loss: 1.5944, Accuracy: 0.3252
Epoch [10/1000], Loss: 1.5920, Accuracy: 0.3205
Epoch [11/1000], Loss: 1.5800, Accuracy: 0.3232
Epoch [12/1000], Loss: 1.5803, Accuracy: 0.3319
Epoch [13/1000], Loss: 1.5786, Accuracy: 0.3256
Epoch [14/1000], Loss: 1.5699, Accuracy: 0.3395
Epoch [15/1000], Loss: 1.5729, Accuracy: 0.3252
Epoch [16/1000], Loss: 1.5631, Accuracy: 0.3355
Epoch [17/1000], Loss: 1.5598, Accuracy: 0.3469
Epoch [18/1000], Loss: 1.5514, Accuracy: 0.3405
Epoch [19/1000], Loss: 1.5574, Accuracy: 0.3383
Epoch [20/1000], Loss: 1.5477, Accuracy: 0.3504
Epoch [21/1000], Loss: 1.5587, Accuracy: 0.3390
E

In [13]:
# Convolutional Neural Network with Attention
cnn_atn_model = CNNAttention(input_size[0],num_classes).to(device)
optimizer = optim.Adam(cnn_atn_model.parameters(), lr=0.0001)
train(cnn_atn_model, train_loaderCNN, criterion, optimizer)

Epoch [1/1000], Loss: 1.8554, Accuracy: 0.2063
Epoch [2/1000], Loss: 1.7011, Accuracy: 0.2804
Epoch [3/1000], Loss: 1.6400, Accuracy: 0.3173
Epoch [4/1000], Loss: 1.5993, Accuracy: 0.3415
Epoch [5/1000], Loss: 1.5681, Accuracy: 0.3465
Epoch [6/1000], Loss: 1.5553, Accuracy: 0.3543
Epoch [7/1000], Loss: 1.5495, Accuracy: 0.3583
Epoch [8/1000], Loss: 1.5315, Accuracy: 0.3633
Epoch [9/1000], Loss: 1.5287, Accuracy: 0.3657
Epoch [10/1000], Loss: 1.5189, Accuracy: 0.3667
Epoch [11/1000], Loss: 1.5192, Accuracy: 0.3722
Epoch [12/1000], Loss: 1.5098, Accuracy: 0.3766
Epoch [13/1000], Loss: 1.5027, Accuracy: 0.3848
Epoch [14/1000], Loss: 1.4923, Accuracy: 0.3800
Epoch [15/1000], Loss: 1.4906, Accuracy: 0.3885
Epoch [16/1000], Loss: 1.4888, Accuracy: 0.3877
Epoch [17/1000], Loss: 1.4829, Accuracy: 0.3854
Epoch [18/1000], Loss: 1.4795, Accuracy: 0.3976
Epoch [19/1000], Loss: 1.4785, Accuracy: 0.3901
Epoch [20/1000], Loss: 1.4726, Accuracy: 0.3944
Epoch [21/1000], Loss: 1.4656, Accuracy: 0.3961
E

In [14]:
# Função para teste
def test(model, test_loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, torch.max(labels, 1)[1])
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == torch.max(labels, 1)[1]).sum().item()

    test_loss = running_loss / len(test_loader)
    test_accuracy = correct / total
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

In [15]:
print("Testing SimpleDNN:")
test(sdnn_model, test_loader, criterion)

Testing SimpleDNN:
Test Loss: 1.4264, Test Accuracy: 0.4533


In [16]:
print("\nTesting LSTMModel:")
test(lstm_model, test_loaderLSTM, criterion)



Testing LSTMModel:
Test Loss: 2.5871, Test Accuracy: 0.4063


In [17]:
print("\nTesting LSTMAttention:")
test(lstm_atn_model, test_loaderLSTM, criterion)


Testing LSTMAttention:
Test Loss: 2.3434, Test Accuracy: 0.4204


In [18]:
print("\nTesting CNNModel:")
test(cnn_model, test_loaderCNN, criterion)



Testing CNNModel:
Test Loss: 1.6494, Test Accuracy: 0.3606


In [19]:
print("\nTesting CNNAttention:")
test(cnn_atn_model, test_loaderCNN, criterion)


Testing CNNAttention:
Test Loss: 1.4693, Test Accuracy: 0.4379
